# Statistical Shape Modeling

Statistical shape models (SSMs) provide a principled way for extracting knowledge from empirically given collections of objects.
SSMs describe the geometric variability in a collection in terms of a mean shape and a hierarchy of principal modes explaining the main trends of shape variation. More precisely, due to the nonlinear structure the mean and the modes are computed in terms of their Riemannian counterparts, viz. Fréchet mean and Pricipal Geodesic Analysis. The resulting models provide a shape prior that can be used to constrain synthesis and analysis problems. In `StatisticalShapeModel` we provide an method to construct an SSM for which the mean agrees with the reference and thus avoids a systematic bias due to choice thereof.

You should build SSMs within this tutorial based on two different Shape Spaces: `PointDistributionModel` and `FundamentalCoords`.
The illustrative example below details how to set up a `FundamentalCoords` SSM and explains its basic usage.

In [ ]:
''' Read and show surfaces. '''

import pyvista as pv

nShapes=4

# load surfaces
meshes = [pv.read(f'./tutorial2_pop_med_image_shape_ana/data/hand{i}.ply') for i in range(1, nShapes + 1)]

# show
pl = pv.Plotter(notebook=True, shape=(1,nShapes))
for i in range(nShapes):
    pl.subplot(0, i)
    pl.add_mesh(meshes[i], smooth_shading=True)
    pl.view_yx()
    pl.camera.roll += 180
    pl.camera.zoom(2)
pl.show(jupyter_backend='static', window_size=(1280,512))

## Task 2 on SSM construction
Choose different shape spaces (FundamentalCoords, PointDistributionModel).
- What can you say about the difference between FCM and PDM, regarding the resulting mean and modes of variation?

Choose different values for `nSteps` and `mode`.

In [ ]:
''' Construct SSM. '''

from morphomatics.geom import Surface
from morphomatics.stats import StatisticalShapeModel
from morphomatics.manifold import FundamentalCoords, PointDistributionModel

# to Surface type
as_surface = lambda mesh: Surface(mesh.points, mesh.faces.reshape(-1, 4)[:, 1:])
surfaces = [as_surface(m) for m in meshes]

# construct model
SSM = StatisticalShapeModel(lambda ref: FundamentalCoords(ref)) # replace me with PointDistributionModel
SSM.construct(surfaces)

print('Done')

Having set up the model one can easily access the mean shape vertex coordinates (`SSM.mean.v`),
as well as the mean coordinates in Shape Space (`SSM.mean_coords`).

In [ ]:
# show mean
pl = pv.Plotter(notebook=True)
pl.add_mesh(pv.PolyData(SSM.mean.v, meshes[0].faces), smooth_shading=True)
pl.view_yx()
pl.camera.roll += 180
pl.show(jupyter_backend='static')

Basic SSM properties can be accessed directly:
* `SSM.modes` its modes of variation,
* `SSM.variances` its per-mode-variances, and
* `SSM.coeffs` its shape coefficients (uniquely determining all input shapes)

Making straightforward use of the above one can generate samples from the SSM along the first mode of variation employing the exponential map of the underlying Shape Space:

In [ ]:
nSteps = 4 # e.g. 4, 8, 12
mode = 2 # e.g. 0, 1, 2

''' sample trajectory along the main mode of variation '''
import numpy as np

# standard deviation associated to kth mode
std = np.sqrt(SSM.variances[mode])

pl = pv.Plotter(notebook=True, shape=(1,nSteps))
for i, t in enumerate(np.linspace(-1.0,1.0,nSteps)):
    # excite mode
    coords = SSM.space.exp(SSM.mean_coords, t * std * SSM.modes[mode])
    # map shape space coords to vertex coords
    v = SSM.space.from_coords(coords)
    # add mesh to plot
    pl.subplot(0, i)
    pl.add_mesh(pv.PolyData(v, meshes[0].faces), smooth_shading=True)
    pl.view_yx()
    pl.camera.roll += 180
    pl.camera.zoom(3)
pl.show(jupyter_backend='static', window_size=(1280,512))

## Task 3 on Synthesis of Shapes from Statistical Shape Model

Rerun the following code cell various times to generate different randomly sampled shapes.
- How would you judge the specificity of the model, i.e. how well do the shythetic shapes fit plausible hand poses?

In [ ]:
# Sample weights from multivariate normal distribution with learned standard deviations
weights = np.random.normal(scale=np.sqrt(SSM.variances))

# Compute tangent vector as blend of principal modes (matrix-vector product)
vec = weights @ SSM.modes

# Shoot geodesic in given direction
coords = SSM.space.exp(SSM.mean_coords, vec)

# map shape space coords to vertex coords
v = SSM.space.from_coords(coords)

# add mesh to plot
pl = pv.Plotter(notebook=True)
pl.add_mesh(pv.PolyData(v, meshes[0].faces))
pl.view_yx()
pl.camera.roll += 180
pl.show(jupyter_backend='ipygany')